In [1]:
# importing all the required modules

from importlib import reload
import nltk
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize, word_tokenize
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from prettytable import PrettyTable
import time
from datetime import timedelta
from sklearn.metrics import recall_score,precision_score,average_precision_score,f1_score
from sklearn.metrics import accuracy_score,roc_auc_score,mean_absolute_error,cohen_kappa_score
from sklearn.preprocessing import label_binarize,LabelEncoder
import statistics
import math
from sklearn.model_selection import KFold,StratifiedKFold
import csv
from collections import Counter
import math
from scipy.spatial import distance
import statistics
from scipy.stats import pearsonr,entropy,mode
from scipy.io import arff
from sklearn.neighbors import NearestNeighbors
import warnings,os,scipy
warnings.filterwarnings('ignore')
from sklearn.base import BaseEstimator, ClassifierMixin



def loadNonIRDatasets(dataset):
    documents,labels=[],[]
    fname=dataset+'.csv'
    with open(fname, 'r') as file:
        reader = csv.reader(file, delimiter = ',')
        for row in reader:
            if len(row)>0:
                if '?' in row:
                    continue
                doc=row[0:len(row)-2]
                try:
                    doc=[abs(float(d)) for d in doc]
                except:
                    continue
                documents.append(doc)
                #print(row)
                labels.append(row[len(row)-1])

    return documents,labels

def Cosine(a, b):#distance
    return distance.cosine(a,b)

def Euclidean(a, b):#distance
    return distance.euclidean(a,b)

def most_common(lst):
    return max(set(lst), key=lst.count)
def PrintDetails(metric="smtp",measure= "Accuracy",Arr=None,kList=[5,15,30,45],train_time=0,test_time=0,std=[]):
    x = PrettyTable()
    #kList=[1,3,5,9,15,30,45]#,70,90,120]
    #if std==[]:
    if len(std)==0:
        x.field_names = ["k",measure,"Train Time","Test time","Total Time"]
    else:
        x.field_names = ["Split",measure,"std","Train Time","Test time","Total Time"]
        
    print("metric",metric)
    print("measure",measure)
    tables.write(("metric\t"+str(metric)+"\n").encode())
    tables.write(("measure\t"+str(measure)+"\n").encode())
    average=0
    total_time=np.add(train_time,test_time)
    for i in range(0,len(Arr)):
        print(train_time[i])
        tr_time=str(timedelta(seconds=(train_time[i])))
        ts_time=str(timedelta(seconds=(test_time[i])))
        
        tot_time=str(timedelta(seconds=(total_time[i])))
        if len(std)==0:
            x.add_row([str(kList[i]),round(Arr[i],4),tr_time,ts_time,tot_time]) 
        else:
            x.add_row([str(kList[i]),round(Arr[i],4),round(std[i],4),tr_time,ts_time,tot_time]) 
            
        average+=Arr[i]
    if len(std)==0:
        x.add_row(["Average",round((average/len(Arr)),4),str(timedelta(seconds=np.mean(train_time))),
                   str(timedelta(seconds=np.mean(test_time))),str(timedelta(seconds=np.mean(total_time)))]) 
        x.add_row(["Std Dev.",round(np.std(Arr),4),np.std(train_time),
                   np.std(test_time),np.std(total_time)])   
    else:
        x.add_row(["Average",round((average/len(Arr)),4),round(np.mean(std),4),str(timedelta(seconds=np.mean(train_time))),
                   str(timedelta(seconds=np.mean(test_time))),str(timedelta(seconds=np.mean(total_time)))]) 
        x.add_row(["Std Dev.",round(np.std(Arr),4),round(np.std(std),4),
                   round(np.std(train_time),10),
                   round(np.std(test_time),10),
                   round(np.std(total_time),10)])
        
    x.padding_width =1
    tables.write(str(x).encode())
    print(x)

class CDNNClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self,n_neighbors=20,distance_metric='euclidean'):
        self.n_neighbors = n_neighbors
        self.distance_metric=distance_metric

    def fit(self, X, y):
        """
        Fit the k-nearest neighbors classifier from the training dataset.
        Parameters
        ----------
        X : {array-like, sparse matrix} of shape (n_samples, n_features)
          Training data.
        y : {array-like, sparse matrix} of shape (n_samples,)
          Target values.
        Returns
        -------
        self : KCDNNClassifier
          The fitted k- Centroid Displacement based nearest neighbors classifier.
        """
        self.X_ = X
        self.y_ = y

        self.fitted_ = True
        # Return the classifier
        return self

    def predict(self, X):
        if self.fitted_ == None:
            raise Exception('predict() called before fit()')
        else:
            input_dim=X.shape[1]
            #calculate distance
            d=scipy.spatial.distance.cdist(X,self.X_,self.distance_metric)
            #get k lowest distance and save to Sx
            indexes=np.argsort(d)[:,:self.n_neighbors] # return k indexes of lowest value in d

            y_pred=[] ##set y_predict list
            for n,index in enumerate(indexes): ##looping through k indexes over the whole test dataset
                Sx = dict()
                for idx in range(self.n_neighbors):
                    key = index[idx]
                    if self.y_[key] in Sx:
                        Sx[self.y_[key]].append(self.X_[key])
                    else:
                        Sx[self.y_[key]] = []
                        Sx[self.y_[key]].append(self.X_[key])

                #calculate current centroids within training dataset
                px = dict()
                for key in Sx:
                    sum_item = np.zeros(input_dim)
                    for i in range(len(Sx[key])):
                        sum_item += Sx[key][i]
                    px_item = sum_item/len(Sx[key])
                    px[key] = px_item

                #calculate new centroid by adding new test data
                qx = dict()
                for key in Sx:
                    sum_item = np.zeros(input_dim)
                    for i in range(len(Sx[key])):
                        sum_item+=Sx[key][i]
                    sum_item += X[n]
                    qx_item = sum_item/(len(Sx[key]) + 1)
                    qx[key] = qx_item

                #calculate displacement
                theta = dict()
                for key in px:
                    if key in qx:
                        theta[key] = np.linalg.norm(px[key] - qx[key])

                label=min(theta, key=theta.get)
                y_pred.append(label)
        return np.array(y_pred)
def classify(k=3,metric="euclidean",termOccurance=None, docOccurance=None):
    accuracy=[]
    precision=[]
    Recall=[]
    fMeasure=[]
    mprecision=[]
    mRecall=[]
    mfMeasure=[]
    roc=[]
    y_pred=[]
    mae=[]
    kappa=[]
    averageMeanPrecison=[]
    func=globals()[metric]
    kList=[5,15,30,45]
    theta=1
    var=np.zeros(train_data.shape[1])
    train_time=[]
    test_time=[]
    M=5
    alpha=0.5
    for k in kList:
        time1=time.time()
        knn=CDNNClassifier(n_neighbors=k).fit(train_data,train_labels)
        time2=time.time()
        pred=knn.predict(test_data)    
        y_pred.append(pred)
        time3=time.time()
        train_time.append(time2-time1)
        test_time.append(time3-time2)
 
   
    class_dict={}
    classes=list(set(labels))
    for i in range(0,len(classes)):
        class_dict[classes[i]]=i
    print(class_dict)
    num_testLabels=[]
    for i in range(0,len(test_labels)):
        num_testLabels.append(class_dict[test_labels[i]])
       
    for i in range(0,len(y_pred)):
        test_labels1=list(test_labels)
        #Accuracy
       
        accuracy.append(accuracy_score(test_labels, y_pred[i]))
        
        #Precison
        precision.append(precision_score(test_labels, y_pred[i],average='macro'))
         
        #Recall
        Recall.append(recall_score(test_labels, y_pred[i],average='macro'))
           
        #f measure
        fMeasure.append(f1_score(test_labels, y_pred[i],average='macro'))
        
        #Precison
        mprecision.append(precision_score(test_labels, y_pred[i],average='micro'))
         
        #Recall
        mRecall.append(recall_score(test_labels, y_pred[i],average='micro'))
           
        #f measure
        mfMeasure.append(f1_score(test_labels, y_pred[i],average='micro'))
        
        kappa.append(cohen_kappa_score(test_labels, y_pred[i]))
      
        test = label_binarize(test_labels1, classes=categories).reshape((-1))
        pred = label_binarize( y_pred[i], classes=categories).reshape((-1))
        
        #roc
        try:
            roc.append(roc_auc_score(test,pred))
        except:
            roc.append(-1)
            
        #Average Mean Precision
        #y_val_true, val_pred = y_val_true.reshape((-1)), val_pred.reshape((-1))
        averageMeanPrecison.append(average_precision_score(test, pred))
        #MAE
        num_pred=[]
        for j in range(0,len(y_pred[i])):
            num_pred.append(class_dict[y_pred[i][j]])
        mae.append(mean_absolute_error(num_testLabels, num_pred))
        
        #PrintDetails(metric=metric,time=str(timedelta(seconds=(time2-time1))),measure="Average Mean Precison",Arr=averageMeanPrecison)


    
   #Accuracy
    PrintDetails(metric=metric,measure="Accuracy",Arr=accuracy,train_time=train_time,test_time=test_time)
    
    #Precison
    PrintDetails(metric=metric,measure="Precision",Arr=precision,train_time=train_time,test_time=test_time)
    #Precison
    PrintDetails(metric=metric,measure="micro Precision",Arr=mprecision,train_time=train_time,test_time=test_time)
    #Recall
    PrintDetails(metric=metric,measure="Recall",Arr=Recall,train_time=train_time,test_time=test_time)
     #Recall
    PrintDetails(metric=metric,measure="micro Recall",Arr=mRecall,train_time=train_time,test_time=test_time)
    #f measure
    PrintDetails(metric=metric,measure="F Measure",Arr=fMeasure,train_time=train_time,test_time=test_time)
    
    #f measure
    PrintDetails(metric=metric,measure="micro F Measure",Arr=mfMeasure,train_time=train_time,test_time=test_time)
    
    #roc
    PrintDetails(metric=metric,measure="ROC",Arr=roc,train_time=train_time,test_time=test_time)
    #Average Mean Precision
    PrintDetails(metric=metric,measure="Average Mean Precison",Arr=averageMeanPrecison,train_time=train_time,test_time=test_time)
    #MAE
    PrintDetails(metric=metric,measure="MAE",Arr=mae,train_time=train_time,test_time=test_time)
  
    accuracy_avg.append(accuracy)
    precision_avg.append(precision)
    recall_avg.append(Recall)
    macroFMeasure_avg.append(fMeasure)
    mprecision_avg.append(mprecision)
    mrecall_avg.append(mRecall)
    mFMeasure_avg.append(mfMeasure)
    roc_avg.append(roc)
    AMP_avg.append(averageMeanPrecison)
    mae_avg.append(mae)
    avg_test_time.append(test_time)
    avg_train_time.append(train_time)
    
import glob
datasets=[]
for file_name in glob.glob("Latest Datasets\\"+'*.csv'):
    file=os.path.split(file_name)[1]
    datasets.append(os.path.splitext(file)[0])

    
for dataset in datasets:
    fname="Latest Datasets\\"+dataset
    print(fname.encode())
    arr,labels=loadNonIRDatasets(fname)
    arr=np.array(arr)
    labels=np.array(labels)
    labels=list(labels)
    le = LabelEncoder()
    labels=le.fit_transform(labels)
    categories=list(set(labels))
    measures=["Euclidean"]
    global w
    w=0
    global Weights
    for met in measures:
        n_split=1
        labels=np.array(labels)
        path = "CDKNN\\April Task1\\Latest Datasets"
        if not os.path.exists(path):
            os.makedirs(path)
        fname=path+'\\table_'+dataset+'_'+met+'_CDKNN.txt'
        tables = open(fname, 'wb')
        k_splits=10
        accuracy_avg=[]
        precision_avg=[]
        recall_avg=[]
        macroFMeasure_avg=[]
        mprecision_avg=[]
        mrecall_avg=[]
        mFMeasure_avg=[]
        roc_avg=[]
        AMP_avg=[]
        mae_avg=[]
        avg_test_time=[]
        avg_train_time=[]
        kf = StratifiedKFold(n_splits=k_splits)
        for trn_ind, tst_ind in kf.split(arr,labels):
            xTrain, xTest = arr[trn_ind], arr[tst_ind]
            yTrain, yTest = labels[trn_ind], labels[tst_ind]
            train_data=np.array(xTrain)
            test_data=np.array(xTest)
            train_labels=np.array(yTrain)
            test_labels=np.array(yTest)
            allData=arr
            termOcc=[]
            docOcc=[]
            tables.write("\n*************".encode())
            tables.write((" Split\t"+str(n_split)).encode())
            tables.write(" ***********\n".encode())
            n_split=n_split+1
            k=1
            print("###############")
            Weights=np.ones(train_data.shape[0])
            classify(k=k,metric=met,termOccurance=termOcc,docOccurance=docOcc)
        list1=[5,15,30,45]
        tables.write("\n************* Average Results ***********\n".encode())
        acc_std=np.std(np.array(accuracy_avg),axis=0)
        accuracy_avg=np.mean(np.array(accuracy_avg),axis=0)
        
        pr_std=np.std(np.array(precision_avg),axis=0)
        precision_avg=np.mean(np.array(precision_avg),axis=0)
        
        mpr_std=np.std(np.array(mprecision_avg),axis=0)
        mprecision_avg=np.mean(np.array(mprecision_avg),axis=0)
        
        rcl_std=np.std(np.array(recall_avg),axis=0)
        recall_avg=np.mean(np.array(recall_avg),axis=0)
        
        mrcl_std=np.std(np.array(mrecall_avg),axis=0)
        mrecall_avg=np.mean(np.array(mrecall_avg),axis=0)
        
        F_std=np.std(np.array(macroFMeasure_avg),axis=0)
        macroFMeasure_avg=np.mean(np.array(macroFMeasure_avg),axis=0)
        
        mF_std=np.std(np.array(mFMeasure_avg),axis=0)
        mFMeasure_avg=np.mean(np.array(mFMeasure_avg),axis=0)
        
        roc_std=np.std(np.array(roc_avg),axis=0)
        roc_avg=np.mean(np.array(roc_avg),axis=0)
        
        amp_std=np.std(np.array(AMP_avg),axis=0)
        AMP_avg=np.mean(np.array(AMP_avg),axis=0)
        
        mae_std=np.std(np.array(mae_avg),axis=0)
        mae_avg=np.mean(np.array(mae_avg),axis=0)
        
        avg_train_time=np.mean(np.array(avg_train_time),axis=0)
        avg_test_time=np.mean(np.array(avg_test_time),axis=0)
        #Accuracy
        PrintDetails(metric=met,measure="Accuracy",Arr=accuracy_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=acc_std)
        
        #Precision
        PrintDetails(metric=met,measure="Precision",Arr=precision_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=pr_std)
        
        #Precision
        PrintDetails(metric=met,measure="Micro Precision",Arr=mprecision_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=mpr_std)


        #Recall
        PrintDetails(metric=met,measure="Recall",Arr=recall_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=rcl_std)
        
         #Recall
        PrintDetails(metric=met,measure="Micro Recall",Arr=mrecall_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=mrcl_std)


        #f measure
        PrintDetails(metric=met,measure="F Measure",Arr=macroFMeasure_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=F_std)
        
       
       
        #f measure
        PrintDetails(metric=met,measure="Micro F Measure",Arr=mFMeasure_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=mF_std)


        #roc
        PrintDetails(metric=met,measure="ROC",Arr=roc_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=roc_std)

        #average Mean precision
        PrintDetails(metric=met,measure="AMP",Arr=AMP_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=amp_std)
         
        #MAE
        PrintDetails(metric=met,measure="MAE",Arr=mae_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=mae_std)
        
        tables.close()
        print("###############")





b'Latest Datasets\\banknote'
###############
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0.0
0.0
0.0
0.0
+----------+----------+------------+-----------------------+-----------------------+
|    k     | Accuracy | Train Time |       Test time       |       Total Time      |
+----------+----------+------------+-----------------------+-----------------------+
|    5     |  0.9565  |  0:00:00   |     0:00:00.011267    |     0:00:00.011267    |
|    15    |  0.942   |  0:00:00   |     0:00:00.012258    |     0:00:00.012258    |
|    30    |  0.9275  |  0:00:00   |     0:00:00.013674    |     0:00:00.013674    |
|    45    |  0.9203  |  0:00:00   |     0:00:00.017484    |     0:00:00.017484    |
| Average  |  0.9366  |  0:00:00   |     0:00:00.013671    |     0:00:00.013671    |
| Std Dev. |  0.0139  |    0.0     | 0.0023618141475825766 | 0.0023618141475825766 |
+----------+----------+------------+-----------------------+-----------------------+
metric Euclidean
measure Precision
0.0
0.0

NameError: name 'kappa_avg' is not defined

In [ ]:
"test"